In [203]:
import pandas as pd
import numpy as np
import os 


In [204]:
cwd = os.getcwd()

In [271]:
dynamic = pd.read_pickle(cwd + '/processed_data/all_data.pkl')

### Vaccine Prompt

In [272]:
dynamic_vaccine = dynamic[['state_name', 'fips', 'Week_end', 'Dose1_Pop_Pct', 'Series_Complete_Pop_Pct',
       'Additional_Doses_Vax_Pct']]
for fips in dynamic_vaccine['fips'].unique():
    
    index = dynamic_vaccine[dynamic_vaccine['fips'] == fips].index
    dynamic_vaccine.loc[index, ['Dose1_Pop_Pct_diff', 'Series_Complete_Pop_Pct_diff', 'Additional_Doses_Vax_Pct_diff']] = \
    dynamic_vaccine[dynamic_vaccine['fips'] == fips].loc[:, ['Dose1_Pop_Pct', 'Series_Complete_Pop_Pct','Additional_Doses_Vax_Pct']].diff().values
dynamic_vaccine = dynamic_vaccine.fillna(0)
dynamic_vaccine[dynamic_vaccine['Dose1_Pop_Pct']>0].sort_values('Week_end')
index1 = dynamic_vaccine[dynamic_vaccine['Dose1_Pop_Pct_diff']<=0.1].index
index2 = dynamic_vaccine[(dynamic_vaccine['Dose1_Pop_Pct_diff']>0.1)&\
                        (dynamic_vaccine['Dose1_Pop_Pct_diff']<=0.37)].index
index3 = dynamic_vaccine[(dynamic_vaccine['Dose1_Pop_Pct_diff']>0.37)&\
                        (dynamic_vaccine['Dose1_Pop_Pct_diff']<=0.85)].index
index4 = dynamic_vaccine[(dynamic_vaccine['Dose1_Pop_Pct_diff']>0.85)].index
dynamic_vaccine.loc[index1, '1_dose_trend'] = 'Stable'
dynamic_vaccine.loc[index2, '1_dose_trend'] = 'Slight Increase'
dynamic_vaccine.loc[index3, '1_dose_trend'] = 'Moderate Increase'
dynamic_vaccine.loc[index4, '1_dose_trend'] = 'Rapid Increase'

index1 = dynamic_vaccine[dynamic_vaccine['Series_Complete_Pop_Pct_diff']<=0.1].index
index2 = dynamic_vaccine[(dynamic_vaccine['Series_Complete_Pop_Pct_diff']>0.1)&\
                        (dynamic_vaccine['Series_Complete_Pop_Pct_diff']<=0.257)].index
index3 = dynamic_vaccine[(dynamic_vaccine['Series_Complete_Pop_Pct_diff']>0.257)&\
                        (dynamic_vaccine['Series_Complete_Pop_Pct_diff']<=0.6)].index
index4 = dynamic_vaccine[(dynamic_vaccine['Series_Complete_Pop_Pct_diff']>0.6)].index
dynamic_vaccine.loc[index1, '2_doses_trend'] = 'Stable'
dynamic_vaccine.loc[index2, '2_doses_trend'] = 'Slight Increase'
dynamic_vaccine.loc[index3, '2_doses_trend'] = 'Moderate Increase'
dynamic_vaccine.loc[index4, '2_doses_trend'] = 'Rapid Increase'

index1 = dynamic_vaccine[dynamic_vaccine['Additional_Doses_Vax_Pct_diff']<=0.1].index
index2 = dynamic_vaccine[(dynamic_vaccine['Additional_Doses_Vax_Pct_diff']>0.1)&\
                        (dynamic_vaccine['Additional_Doses_Vax_Pct_diff']<=0.2)].index
index3 = dynamic_vaccine[(dynamic_vaccine['Additional_Doses_Vax_Pct_diff']>0.2)&\
                        (dynamic_vaccine['Additional_Doses_Vax_Pct_diff']<=0.85)].index
index4 = dynamic_vaccine[(dynamic_vaccine['Additional_Doses_Vax_Pct_diff']>0.85)].index
dynamic_vaccine.loc[index1, 'booster_trend'] = 'Stable'
dynamic_vaccine.loc[index2, 'booster_trend'] = 'Slight Increase'
dynamic_vaccine.loc[index3, 'booster_trend'] = 'Moderate Increase'
dynamic_vaccine.loc[index4, 'booster_trend'] = 'Rapid Increase'

In [273]:
for index, row in dynamic_vaccine.iterrows():
    dose1 = str(int(row['Dose1_Pop_Pct']))[:2]
    dose2 = str(int(row['Series_Complete_Pop_Pct']))[:2]
    booster = str(int(row['Additional_Doses_Vax_Pct']))[:2]
    
    prompt = 'To date, ' + dose1 + '% of population got at least one vaccine dose with a ' \
    + row['1_dose_trend'] + ' trend, ' +\
    dose2 + '% were fully vaccinated with a ' + row['2_doses_trend'] + ' trend, ' + \
    'and ' + booster + '% got booster with a ' + row['booster_trend'] + ' trend.'
    
    dynamic_vaccine.loc[index, 'Vaccine_prompt'] = prompt

In [274]:
dynamic = dynamic.merge(dynamic_vaccine[['state_name', 'fips', 'Week_end', 'Vaccine_prompt']])

### Previous infection prompt

In [275]:
dynamic_inf = dynamic[['state_name', 'fips', 'Week_end', 'previous_infection_12w']]
for fips in dynamic_inf['fips'].unique():
    
    index = dynamic_inf[dynamic_inf['fips'] == fips].index
    dynamic_inf.loc[index, ['previous_infection_12w_diff']] = \
    dynamic_inf[dynamic_inf['fips'] == fips].loc[:, 'previous_infection_12w'].diff().values
        
dynamic_inf = dynamic_inf.fillna(0)

index1 = dynamic_inf[dynamic_inf['previous_infection_12w_diff']<=-0.2].index
index2 = dynamic_inf[(dynamic_inf['previous_infection_12w_diff']>-0.2)&\
                        (dynamic_inf['previous_infection_12w_diff']<=0)].index
index3 = dynamic_inf[(dynamic_inf['previous_infection_12w_diff']>0)&\
                        (dynamic_inf['previous_infection_12w_diff']<=0.1)].index
index4 = dynamic_inf[(dynamic_inf['previous_infection_12w_diff']>0.1)].index
dynamic_inf.loc[index1, 'prev_inf_trend'] = 'Rapid Decrease'
dynamic_inf.loc[index2, 'prev_inf_trend'] = 'Moderate Decrease'
dynamic_inf.loc[index3, 'prev_inf_trend'] = 'Moderate Increase'
dynamic_inf.loc[index4, 'prev_inf_trend'] = 'Rapid Increase'

for index, row in dynamic_inf.iterrows():
    pre_inf = str(round(row['previous_infection_12w'], 1))
    prompt = 'Around ' \
    + pre_inf + \
    '% of the population reported infections over the past three months, the population immunity is ' +\
    row['prev_inf_trend'] + '.'
    dynamic_inf.loc[index, 'Previous_infection_prompt'] = prompt
    
dynamic = dynamic.merge(dynamic_inf[['state_name', 'fips', 'Week_end', 'Previous_infection_prompt']])

### Policy Prompt

In [276]:
policy = dynamic[['fips', 'Week_end','C1M_School closing',
       'C2M_Workplace closing', 'C3M_Cancel public events',
       'C4M_Restrictions on gatherings', 'C6M_Stay at home requirements',
       'C7M_Restrictions on internal movement',
       'C8NV_International travel controls',
       'C8V_International travel controls', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing', 'H6M_Facial Coverings',
       'H8M_Protection of elderly people']]
policy['C1M_School closing'] = policy['C1M_School closing'].replace('require closing all levels', 
                                                                    'require closing all')
policy['C1M_School closing'] = policy['C1M_School closing'].replace('no measures', 
                                                                    'no restrictions')
policy['C1M_School closing'] = policy['C1M_School closing'].replace('recommend closing or all schools open with alterations resulting in significant differences compared to non-Covid-19 operations', 
                                     'recommend closing')
policy['C1M_School closing'] = policy['C1M_School closing'].replace('require closing (only some levels or categories, eg just high school, or just public schools)', 
                                     'require closing some')

policy['C2M_Workplace closing'] = policy['C2M_Workplace closing'].replace('require closing (or work from home) for all-but-essential workplaces (eg grocery stores, doctors)', 
                                                                          'require closing all')
policy['C2M_Workplace closing'] = policy['C2M_Workplace closing'].replace('no measures', 
                                                                          'no restrictions')
policy['C2M_Workplace closing'] = policy['C2M_Workplace closing'].replace('recommend closing (or recommend work from home) or all businesses open with alterations resulting in significant differences compared to non-Covid-19 operation', 
                                     'recommend closing')
policy['C2M_Workplace closing'] = policy['C2M_Workplace closing'].replace('require closing (or work from home) for some sectors or categories of workers', 
                                     'require closing some')

policy_select = policy[['fips', 'Week_end', 'C1M_School closing', 'C2M_Workplace closing',
       'C4M_Restrictions on gatherings',
       'H8M_Protection of elderly people']]
policy_select['H8M_Protection of elderly people'].unique()
policy_select['H8M_Protection of elderly people'] = policy_select['H8M_Protection of elderly people'].replace('Narrow restrictions for isolation, hygiene in LTCFs, some limitations on external visitors and/or restrictions protecting elderly people at home',
                                                         'Narrow restrictions for isolation, some limitations on external visitors.')

policy_select['H8M_Protection of elderly people'] = policy_select['H8M_Protection of elderly people'].replace('Recommended isolation, hygiene, and visitor restriction measures in LTCFs and/or elderly people to stay at home',
                                                         'Recommended isolation, and visitor restriction.')

policy_select['H8M_Protection of elderly people'] = policy_select['H8M_Protection of elderly people'].replace('Extensive restrictions for isolation and hygiene in LTCFs, all non-essential external visitors prohibited, and/or all elderly people required to stay at home and not leave the home with minimal exceptions, and receive no external visitors',
                                                         'Extensive restrictions for isolation and all non-essential external visitors prohibited.')

for col in ['C1M_School closing', 'C2M_Workplace closing',
            'C4M_Restrictions on gatherings']:
    for fips in policy_select['fips'].unique():
        index = policy_select[policy_select['fips'] == fips].index
        
        policy_select.loc[index, col + '_lag1'] = policy_select[policy_select['fips'] == fips][col].shift(1)

policy_select = policy_select.fillna(method='bfill')

/var/folders/fg/qtvw7sp54r91rm2skfzzb8ww0000gn/T/ipykernel_30162/858000607.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  policy['C1M_School closing'] = policy['C1M_School closing'].replace('require closing all levels',
/var/folders/fg/qtvw7sp54r91rm2skfzzb8ww0000gn/T/ipykernel_30162/858000607.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  policy['C1M_School closing'] = policy['C1M_School closing'].replace('no measures',
/var/folders/fg/qtvw7sp54r91rm2skfzzb8ww0000gn/T/ipykernel_30162/858000607.p

In [277]:
for index, row in policy_select.iterrows():
    if (row['C1M_School closing'] == row['C1M_School closing_lag1']) and\
    (row['C2M_Workplace closing'] == row['C2M_Workplace closing_lag1']) and\
    (row['C4M_Restrictions on gatherings'] == row['C4M_Restrictions on gatherings_lag1']):
        
#         prompt = 'All containment and closure policies remain the same as previous week '
        prompt = '' + \
        row['C1M_School closing'] + ' for school policy, ' + row['C2M_Workplace closing'] +\
        ' for workplace, ' + row['C4M_Restrictions on gatherings'] + ' for gatherings.'
        
        policy_select.loc[index, 'policy_prompt_1'] = prompt
        
    else:
        
        prompt = ''
#         prompt = 'There are some changes in the containment and closure policies. '
        
        if row['C1M_School closing'] != row['C1M_School closing_lag1']:
            prompt_school = 'For school policy change from ' + row['C1M_School closing_lag1'] +\
            ' to ' + row['C1M_School closing'] + '. '
        else:
            prompt_school = 'School policy remains as ' + row['C1M_School closing'] + '. '
            
        if row['C2M_Workplace closing'] != row['C2M_Workplace closing_lag1']:
            prompt_workplace = 'For workplace change from ' + row['C2M_Workplace closing_lag1'] +\
            ' to ' + row['C2M_Workplace closing'] + '. '
        else:
            prompt_workplace = 'Workplace policy remains as ' + row['C1M_School closing'] + '. '
            
        if row['C4M_Restrictions on gatherings'] != row['C4M_Restrictions on gatherings_lag1']:
            prompt_gathering = 'For gatherings change from ' + row['C4M_Restrictions on gatherings_lag1'] +\
            ' to ' + row['C4M_Restrictions on gatherings'] + '. '
        else:
            prompt_gathering = 'Gatherings policy remains as ' + row['C4M_Restrictions on gatherings'] + '. '
            
        prompt = prompt + prompt_school + prompt_workplace + prompt_gathering
        
        policy_select.loc[index, 'policy_prompt_1'] = prompt
            
            



    

In [278]:
policy_select['policy_elderly'] = 'For elderly patients, ' + policy_select['H8M_Protection of elderly people']
policy_select['policy_prompt'] = policy_select['policy_prompt_1'] + ' ' + policy_select['policy_elderly']
dynamic = dynamic.merge(policy_select[['fips', 'Week_end', 'policy_prompt']])
dynamic['policy_prompt'] = dynamic['policy_prompt'].str.capitalize()

In [279]:
static = pd.read_pickle(cwd + '/processed_data/static_data.pkl')

In [280]:
dynamic = dynamic.merge(static)

In [281]:
cases_gpt_trend = pd.read_pickle(cwd + '/processed_data/cases_gpt_trend.pkl')
hosp_gpt_trend = pd.read_pickle(cwd + '/processed_data/hosp_gpt_trend.pkl')

In [282]:
dynamic['hospitalization_per_100k_gpt_trend'] = hosp_gpt_trend['hospitalization_per_100k_gpt_trend']
dynamic['reported_cases_per_100k_gpt_trend'] = cases_gpt_trend['reported_cases_per_100k_gpt_trend']

In [287]:
prompt_augmentation = pd.read_pickle(cwd + '/processed_data/dynamic_prompt.pkl')

In [284]:
dynamic = dynamic.merge(prompt_augmentation)

In [285]:
df_final = dynamic[['state_name', 'fips', 'Week', 'Week_start', 'Week_end',
       'hospitalization_per_100k', 'hospitalization_per_100k_sm',
       'reported_cases_per_100k', 'Dose1_Pop_Pct', 'Series_Complete_Pop_Pct',
       'Additional_Doses_Vax_Pct', 'Abs_Change', 'Abs_Trend',
       'reported_cases_per_100k_gpt_trend',
       'hospitalization_per_100k_gpt_trend', 'Trend_prompt', 'dynamic_prompt',
       'dynamic_prompt_v2', 'dynamic_prompt_v3', 
       'Population', 'under_20', 'over_65', 'White', 'Black', 'Multiple_race',
       'Not_Hispanic', 'Hispanic', 'medicaid_coverage', 'medicare_coverage',
       'uninsured_percent', 'medicaid_spending',
       'private_health_insurance_spending', 'medicare_spending_by_residence',
       'health_care_spending', 'healthcare_utilization', 'poor_health_status',
       'adults_at_high_risk', 'poverty_rate', 'social_vulnerability_index',
       'Healthcare Access and Quality Index', 'Older_at_high_risk',
       'dem_percent', 'rep_percent', 'other_percent', 'State',
       'Static_description', 'Static_description_v2', 'Static_description_v3',
       'Static_description_v4', 'Static_description_v5', 'Abs_Trend_target_1w', 'Abs_Trend_target_3w']]

In [270]:
df_final.to_pickle(cwd + '/processed_data/final_data.pkl')